# Slices, stacks

In [ ]:
import numpy             as np
import scipy.linalg      as la
import cv2               as cv
import skimage           as si
import skimage.io        as io
import matplotlib.pyplot as plt

%matplotlib inline

## numpy indexing,  slicing, stacking

In [ ]:
x = 100*np.arange(10)

x

In [ ]:
x[:3]

In [ ]:
x[3:-2]

In [ ]:
x[[4,4,2,5]]

En el siguiente ejemplo intercambiamos el orden de los canales de color.

In [ ]:
from skimage import data

img = data.astronaut()
print(img.shape)

plt.imshow(np.hstack([img,img[:,:,[2,1,0]]]));

In [ ]:
plt.imshow(  np.sum(img,axis=2)  ,'gray');

Podemos convertir a monocromo con una **contracción** de arrays:

In [ ]:
plt.imshow(  img @ np.array([0.3, 0.5, 0.2])  ,'gray');

In [ ]:
(img @ np.array([0.3, 0.5, 0.2])).shape

In [ ]:
(img @ np.array([0.3, 0.5, 0.2])).dtype

In [ ]:
img.dtype

## Troceado de una imagen

In [ ]:
path = "http://robot.inf.um.es/material/va/images/"

img = io.imread(path+"palmeras.jpg")
print(img.shape)

tile = 50
r,c,_ = img.shape
rt,ct = r//tile, c//tile

# recortamos la imagen para que las piezas encajen perfectamente
img = img[:rt*tile,:ct*tile,:]
plt.imshow(img);
print(img.shape)

Descomposición en rectángulos usando `np.split`:

In [ ]:
# otra posibilidad más fea
# piezas = [[rot(img[j*tile:(j+1)*tile,(k*tile):(k+1)*tile,:]) for k in range(ct)] for j in range(rt)]

piezas = [np.split(x, ct, axis=1) for x in np.split(img,rt)]
piezas = sum(piezas,[])  # concatenamos la lista de listas generadas

print(rt,ct,len(piezas))

In [ ]:
plt.imshow(piezas[74]);

Para recomponer las piezas definimos una función auxiliar `chunks` para agrupar los elementos de una lista.

In [ ]:
# esta versión produce una lista de listas
# def chunks(l, n):
#     return [ l[i:i+n] for i in range(0, n*(len(l)//n), n) ]

# esta versión produce un generador
def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i+n]
        
list(chunks(list(range(12)),3))

Aplicamos `np.hstack` para pegar en horizontal cada lista de trozos, y luego `np.vstack` para juntar en vertical todas las bandas creadas.

In [ ]:
plt.imshow(np.vstack(list(map(np.hstack,chunks(piezas,ct)))));

Ahora vamos a revolver los trozos. La siguiente función aplica una rotación aleatoria en una de las 4 orientaciones:

In [ ]:
def rot(x):
    return np.rot90(x,np.random.randint(4))

In [ ]:
piezas= list(map(rot, piezas))
piezas = np.random.permutation(piezas)

puzz = np.vstack(list(map(np.hstack,chunks(piezas,ct))))
plt.imshow(puzz);
#io.imsave('kk.png',puzz)